# Lectures 13 to 15
- Good to have some extra tools in SQL
- Connecting DuckDB to Python
- 13 subquery
- 14 views
- 15 cte
* 16 windows functions will be covered in Data Modelling


## Connect to a database and read the ingestion script
- Open a file with `with open` and we need a PATH to ingest the ingestion.sql
- Create or uide the path, read and save the fie with the named variable `sql_code`

- This code opens the SQL script file at sql/ingestion.sql, reads all its text into a variable called sql_code, and then prints that text so you can see the SQL contents. The with open(...) part is a safe way to work with files—it automatically closes the file when it’s done. If the file path is incorrect or the file doesn’t exist, Python will raise an error; otherwise, you’ll see the full SQL script in the notebook output.

In [1]:
with open("sql/ingestion.sql", "r") as sql_script:
    sql_code = sql_script.read()

print(sql_code)

CREATE SCHEMA IF NOT EXISTS staging;

CREATE TABLE IF NOT EXISTS staging.students AS (
    SELECT * FROM read_csv_auto('data/students.csv')
);

CREATE TABLE IF NOT EXISTS staging.teachers AS (
    SELECT * FROM read_csv_auto('data/teachers.csv')
);

CREATE TABLE IF NOT EXISTS staging.housing_sales AS (
    SELECT * FROM read_csv_auto('data/housing_sales.csv')
);

CREATE TABLE IF NOT EXISTS staging.neighborhoods AS (
    SELECT * FROM read_csv_auto('data/neighborhoods.csv')
);

CREATE TABLE IF NOT EXISTS staging.categories AS (
    SELECT * FROM read_csv_auto('data/categories.csv')
);

CREATE TABLE IF NOT EXISTS staging.sales AS (
    SELECT * FROM read_csv_auto('data/sales.csv')
);


## Open up a database connection with Duckdb
This cell connects to a local DuckDB database file at `data/lecture_13to15`.duckdb using a context manager (with ... as conn:). The context manager ensures the connection is opened at the start and safely closed when the block finishes, even if an error occurs. Inside the block, it runs sql_code, which you previously loaded from an ingestion script; that executes whatever SQL is needed to create or populate tables in the database.

Next, it runs DESC; to ask DuckDB for a description of the database objects (typically tables/columns), and .df() converts that result into a pandas DataFrame. The DataFrame is assigned to describe, and the final line displays describe so you can see a readable summary of the database schema in your notebook.

`with` statements handles the connection and closes it down properly. We cannot run anything outside the databas because it will be closed automatically.


In [2]:
import duckdb

with duckdb.connect('data/lecture_13to15.duckdb') as conn:
    conn.execute(sql_code)
    describe = conn.execute("DESC;").df()

describe

,database,schema,name,column_names,column_types,temporary
0,lecture_13to15,staging,categories,"[category_id, category_name, parent_category_id]","[BIGINT, VARCHAR, BIGINT]",False
1,lecture_13to15,staging,housing_sales,"[sale_id, neighborhood_id, address, date_of_sa...","[VARCHAR, BIGINT, VARCHAR, DATE, VARCHAR, BIGI...",False
2,lecture_13to15,staging,neighborhoods,"[neighborhood_id, name, city, postal_code, med...","[BIGINT, VARCHAR, VARCHAR, BIGINT, BIGINT, BIG...",False
3,lecture_13to15,staging,sales,"[product_id, product_name, category_id, quanti...","[VARCHAR, VARCHAR, BIGINT, BIGINT, BIGINT]",False
4,lecture_13to15,staging,students,"[student_id, grade_math, class_name]","[VARCHAR, BIGINT, VARCHAR]",False
5,lecture_13to15,staging,teachers,"[class_name, teacher_name]","[VARCHAR, VARCHAR]",False


## Create a connection to the database
but we dont want to type out the `with` statement all the time.

In [3]:
# duckdb.sql("FROM staging.students;") <- Does not work as connection is closed

In [4]:
import pandas as pd

def query_database(query: str, db_path: str = "data/lecture_13to15.duckdb") -> pd.DataFrame:
    """Conveniece function to open a DuckDB connection, run queries, and return a DataFrame."""
    with duckdb.connect(db_path) as conn:
        return conn.execute(query=query).df()

query_database("FROM staging.students;")

,student_id,grade_math,class_name
0,S001,92,A
1,S002,85,A
2,S003,78,B
3,S004,88,B
4,S005,95,C
5,S006,67,C
6,S007,81,D
7,S008,74,D
8,S009,89,E
9,S010,93,E


In [6]:
from utils import query_database

query_database("FROM staging.students;").head()

,student_id,grade_math,class_name
0,S001,92,A
1,S002,85,A
2,S003,78,B
3,S004,88,B
4,S005,95,C


## Lecture 13: Subquery
- Find all rows that grade is higher than the average
- Notice that the query to find out the average becomes a subquery in the 2nd code block below.

In [ ]:
# Find out the average grade in math
query_database("""--sql
    SELECT AVG(grade_math)
    FROM staging.students;
""")

,avg(grade_math)
0,84.2


In [9]:
# Use the query above as a subquery to find all students with above average math grades
query_database("""--sql
    SELECT * 
    FROM staging.students
    WHERE grade_math > (
        SELECT AVG(grade_math)
        FROM staging.students);
""")


,student_id,grade_math,class_name
0,S001,92,A
1,S002,85,A
2,S004,88,B
3,S005,95,C
4,S009,89,E
5,S010,93,E


## Lecture 14: Views
## Lecture 14: Views (Beginner Explanation)

| Concept | Simple Explanation |
|------|-------------------|
| What is a VIEW? | A **VIEW** is a saved SQL query that behaves like a table. You can select from it, but it does not store data itself. |
| Is data stored? | No. A VIEW does **not** store data. Each time you use it, the database **re-runs the query** behind the view. |
| Why use a VIEW? | To make complex queries easier to reuse and easier to understand. |
| Virtual table | A VIEW is often called a **virtual table** because it looks like a table but is built from a query. |
| Simplifying queries | Instead of writing a long SQL query every time, you can query the VIEW using a simple `SELECT * FROM view_name`. |
| Security / access control | A VIEW can hide sensitive columns (for example, salary or personal data) from end users. |
| User-friendly interface | Views provide a clean, simple interface for analysts or applications without exposing raw tables. |
| Performance note | Since the query runs every time, very complex views may be slower than real tables. |
| Example usage | `SELECT * FROM customer_summary_view;` |


In [10]:
query_database("""--sql
    FROM staging.housing_sales;
""")

,sale_id,neighborhood_id,address,date_of_sale,property_type,rooms,living_area_sqm,year_built,sale_price_usd
0,S-24001,1,124 Maple Crest Ave,2024-10-09,Apartment,2,58,2018,245575
1,S-24002,1,9 Birch Hollow Ln,2025-02-21,Townhouse,4,112,2006,454575
2,S-24003,2,77 Riverwalk Way,2025-03-18,Detached,5,168,1999,757625
3,S-24004,2,18 Marina Bluff Ct,2025-07-05,Apartment,3,86,2015,412775
4,S-24005,3,401 Quarry Gate Rd,2024-12-12,Semi-Detached,4,124,1987,381425
5,S-24006,3,15 Basalt Spur,2025-05-27,Loft,2,72,2020,297825
6,S-24007,4,990 Vista Crest Dr,2025-01-16,Detached,6,212,2003,883025
7,S-24008,4,44 Lantern Ridge Pl,2025-08-11,Apartment,2,61,2017,287375
8,S-24009,5,223 Lakeview Promenade,2024-11-03,Condo,3,88,2012,370975
9,S-24010,5,7 Echo Bay Mews,2025-04-22,Townhouse,4,118,2010,485925


- Pick `property_type` and put into a `view` to give to downstream users
- If storing this as a table and the underlying data changes (eg. more houses are added), then the `average sales table` will be wrong.
- With `view`, it will run the sql query when the downstream user is querying the view, which gives fresh results all the time.


In [15]:
query_database("""--sql
    CREATE OR REPLACE VIEW staging.avg_price_type AS
    SELECT
        property_type,
        AVG(sale_price_usd):: INT AS avg_price
    FROM staging.housing_sales
    GROUP BY property_type;
""")

,Count


In [16]:
# Query the view
query_database("""--sql
    SELECT * 
    FROM staging.avg_price_type;
""")

,property_type,avg_price
0,Loft,316112
1,Apartment,290361
2,Townhouse,477913
3,Condo,312803
4,Detached,699105
5,Semi-Detached,391875
6,Bungalow,423225
7,Terraced,350075


In [17]:
query_database("""--sql
SELECT *
FROM information_schema.views
WHERE table_catalog = 'lecture_13to15';
               """)

,table_catalog,table_schema,table_name,view_definition,check_option,is_updatable,is_insertable_into,is_trigger_updatable,is_trigger_deletable,is_trigger_insertable_into
0,lecture_13to15,staging,avg_price_type,CREATE VIEW staging.avg_price_type AS SELECT p...,NONE,NO,NO,NO,NO,NO


In [18]:
query_database("""--sql
SELECT *
FROM information_schema.tables
WHERE table_catalog = 'lecture_13to15';
               """)

,table_catalog,table_schema,table_name,table_type,self_referencing_column_name,reference_generation,user_defined_type_catalog,user_defined_type_schema,user_defined_type_name,is_insertable_into,is_typed,commit_action,TABLE_COMMENT
0,lecture_13to15,staging,categories,BASE TABLE,None,None,None,None,None,YES,NO,None,None
1,lecture_13to15,staging,housing_sales,BASE TABLE,None,None,None,None,None,YES,NO,None,None
2,lecture_13to15,staging,neighborhoods,BASE TABLE,None,None,None,None,None,YES,NO,None,None
3,lecture_13to15,staging,sales,BASE TABLE,None,None,None,None,None,YES,NO,None,None
4,lecture_13to15,staging,students,BASE TABLE,None,None,None,None,None,YES,NO,None,None
5,lecture_13to15,staging,teachers,BASE TABLE,None,None,None,None,None,YES,NO,None,None
6,lecture_13to15,staging,avg_price_type,VIEW,None,None,None,None,None,NO,NO,None,None


## CTE: Common Table Expression (Beginner Explanation)

| Concept | Simple Explanation |
|------|-------------------|
| What is a CTE? | A **CTE (Common Table Expression)** is a temporary named result set created using the `WITH` keyword. |
| When does it exist? | A CTE exists **only during the execution of the query**. It is not saved in the database. |
| Why use a CTE? | To make long or complex SQL queries easier to read, understand, and maintain. |
| Temporary result | The result of a CTE can be reused multiple times within the same query. |
| Improves readability | CTEs let you break a complex query into logical steps instead of nesting subqueries. |
| Alternative to subqueries | A CTE often replaces deeply nested subqueries, making SQL cleaner and more readable. |
| Scope | A CTE can only be referenced by the query that immediately follows it. |
| Example structure | `WITH temp_table AS (SELECT ...) SELECT * FROM temp_table;` |
| Common use cases | Data cleaning, aggregations, filtering, and step-by-step transformations. |


In [19]:
query_database("""--sql
    FROM staging.sales;
""")

,product_id,product_name,category_id,quantity,unit_price_sek
0,P-0001,"Ultrabook 13\"" 8GB/256GB",102,2,11990
1,P-0002,"Gaming Laptop 15\"" 16GB/512GB",102,1,15990
2,P-0003,Office Desktop i5,103,3,7990
3,P-0004,"All-in-One PC 24\""",103,1,9990
4,P-0005,"Android Tablet 10\""",104,2,3490
...,...,...,...,...,...
95,P-0096,Dark Chocolate 85% 100g,160,5,29
96,P-0097,Herbal Tea 40 bags,160,3,49
97,P-0098,"Student Laptop 14\"" 8GB",102,2,6990
98,P-0099,Office Desktop i7,103,1,10990


In [27]:
query_database("""--sql
WITH avg_sales AS (
    SELECT
        category_id,
        AVG(quantity * unit_price_sek):: INT AS avg_sales_sek
    FROM staging.sales
    GROUP BY category_id)
SELECT 
    c.category_id,
    avg_sales_sek
FROM avg_sales a
JOIN staging.categories c ON c.category_id = a.category_id
ORDER BY avg_sales_sek DESC;
""")

,category_id,avg_sales_sek
0,102,18820
1,103,15485
2,105,7189
3,111,3240
4,104,2929
5,100,2790
6,112,2590
7,121,1563
8,122,1059
9,110,1035
